In [1]:
from langchain_community.document_loaders import TextLoader
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
import os




In [2]:
os.environ["OPENAI_API_BASE"] = "https://openrouter.ai/api/v1"
os.environ["OPENAI_API_KEY"] = "Your Own API Key"

model = ChatOpenAI(model="deepseek/deepseek-r1:free")

# STEP 1A - Indexing (Document loader or Document ingestion)

In [3]:
loader = TextLoader('russian_test.txt', encoding='utf-8')

docs = loader.load()
texts = [doc.page_content for doc in docs]

In [4]:
texts

['Привет всем! Я Никита. Я Марина, привет! Мы ведем подкаст «Easy Russian» от Easy Languages, \nподкаст для тех, кто изучает русский язык, несмотря ни на что. Это седьмой выпуск нашего подкаста, \nв котором мы поговорим о том, как война изменила русский язык. Ну, не случайно мы говорим об этом \nименно сейчас. Все, конечно, заняты этой страшной годовщиной, которая приближается. К моменту выхода \nподкаста до нее останется всего два дня, если я не ошибаюсь. Приближается двадцать четвертое февраля, \nдень, когда Россия начала эту страшную, бессмысленную и очень затяжную войну, которая, мне кажется, \nизменила жизнь всех. Да, она, конечно, изменила и язык тоже. И мы поговорим о том, какие новые слова появились. \nИ как в принципе язык отражает то, что происходит в стране и в мире. Да, в общем, Никита, как ты уже сказал, \nдвадцать четвертое февраля две тысячи двадцать второго года. Точно... Ну, почти год назад началась война, Россия \nнапала на Украину, и в тот момент я был в Люксембурге 

# STEP 1b - Indexing (Text Splitter)

In [5]:
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
chunks = splitter.create_documents(texts)

print(len(chunks))

print(f'\nContent: {chunks[0].page_content}\n')

# for i, chunks in enumerate(chunks):
#     print(f'Document {i+1}')
#     print(f'\nContent: {chunks.page_content}\n')

11

Content: Привет всем! Я Никита. Я Марина, привет! Мы ведем подкаст «Easy Russian» от Easy Languages, 
подкаст для тех, кто изучает русский язык, несмотря ни на что. Это седьмой выпуск нашего подкаста, 
в котором мы поговорим о том, как война изменила русский язык. Ну, не случайно мы говорим об этом 
именно сейчас. Все, конечно, заняты этой страшной годовщиной, которая приближается. К моменту выхода 
подкаста до нее останется всего два дня, если я не ошибаюсь. Приближается двадцать четвертое февраля,



# STEP 1c - Indexing (Embedding Generation and storing in vector store)

In [6]:
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={"trust_remote_code": True}
)
vector_store = FAISS.from_documents(
    documents=chunks,
    embedding = embeddings
)

# add documents in the vector store
added_vector_store = vector_store.add_documents(chunks)

In [7]:
added_vector_store

['6a1cb151-c304-4691-8ddd-0255c6a69c9c',
 'e7f28178-31ce-482d-a5db-ef2163cdffbb',
 'beea7cc8-3baf-4c0f-825a-3c07325b6d02',
 '49eea65a-bded-40cd-92d2-b162ac6cb120',
 'f33e9c2f-6042-47ec-97aa-df645f3ac8b5',
 '5fa69ead-612a-4de8-856c-d9043658d46d',
 '4307f7fe-e283-45f5-8125-54b2b0882021',
 'e5f2fc23-ac26-4854-b75b-eb8d9f10ac65',
 '42f368e2-ec5f-451b-8ac2-686d14f41b20',
 '1df86f61-07e6-4701-8f09-fd6d0b46b3a3',
 '834a8122-92cb-4ac1-b08e-a8a0c9e0f039']

In [8]:
len(added_vector_store)

11

In [9]:
vector_store.get_by_ids(['8ca1e6a4-3586-40e0-9a4c-6e0652dc481c'])

[]

# Step 2 - Retriever

In [10]:
retriever = vector_store.as_retriever(search_type='similarity', search_kwargs={'k':4})

In [11]:
retriever.invoke('what are benefits of technology?')

[Document(id='66b01121-d081-4822-ab58-b8ceb39891f8', metadata={}, page_content='этой парадигмы было очень большое деление на друга или врага.'),
 Document(id='834a8122-92cb-4ac1-b08e-a8a0c9e0f039', metadata={}, page_content='этой парадигмы было очень большое деление на друга или врага.'),
 Document(id='43bc8391-4ca2-4b1a-8059-62f406f5e370', metadata={}, page_content='протяжении многих лет такие слова, как «довоенный», «военный» и «послевоенный», всегда автоматически переносили \nнас в сороковые годы прошлого века. Довоенные песни, послевоенная мода, я не знаю, что угодно. Все, произошло \nвнезапное обновление, сегодня везде, уже год я читаю, слушаю, смотрю разных людей, которые используют слова \nдовоенный, послевоенный, военный. Ну, послевоенный, мы, к сожалению, еще не достигли, хотя иногда люди говорят об'),
 Document(id='42f368e2-ec5f-451b-8ac2-686d14f41b20', metadata={}, page_content='протяжении многих лет такие слова, как «довоенный», «военный» и «послевоенный», всегда автоматиче

# Step 3 - Augmentation

In [12]:
prompt = PromptTemplate(
    template='''
You are a helpful assistant.
answer ONLY from the provided transcript context.
if the context is insufficient, just say you do not know.
provide response in given language {language}

{context}
question: {question}
    ''',
    input_variables=['context', 'question'],
    partial_variables={"language": "russian"}
)

In [13]:
question = 'is the topic of phones is discussed in this video? if yes then what was discussed'
language = 'russian'
retrieved_docs = retriever.invoke(question)

In [14]:
retrieved_docs

[Document(id='d0045684-1f83-4497-ba82-1540b322b061', metadata={}, page_content='культурой, людьми и взглядами. И война, конечно... Господи, ну кто бы... Часть культуры. И, кстати, я подхвачу \nвашу мысль. Война действительно является частью этой культуры, и это, к сожалению, та часть, которая стала очень \nважной для российского общества и для его языка. И в целом она формирует. Главную, формирующую, на мой взгляд, \nдвижущую силу развития русского языка, ну, я просто все время думаю об этом с... со странным чувством, что на'),
 Document(id='e5f2fc23-ac26-4854-b75b-eb8d9f10ac65', metadata={}, page_content='культурой, людьми и взглядами. И война, конечно... Господи, ну кто бы... Часть культуры. И, кстати, я подхвачу \nвашу мысль. Война действительно является частью этой культуры, и это, к сожалению, та часть, которая стала очень \nважной для российского общества и для его языка. И в целом она формирует. Главную, формирующую, на мой взгляд, \nдвижущую силу развития русского языка, ну, я 

In [15]:
context_text

NameError: name 'context_text' is not defined

In [ ]:
final_prompt = prompt.invoke({'context':context_text, 'question': question, 'language': language})

In [ ]:
final_prompt

StringPromptValue(text='\nYou are a helpful assistant.\nanswer ONLY from the provided transcript context.\nif the context is insufficient, just say you do not know.\nprovide response in given language russian\n\nкультурой, людьми и взглядами. И война, конечно... Господи, ну кто бы... Часть культуры. И, кстати, я подхвачу \nвашу мысль. Война действительно является частью этой культуры, и это, к сожалению, та часть, которая стала очень \nважной для российского общества и для его языка. И в целом она формирует. Главную, формирующую, на мой взгляд, \nдвижущую силу развития русского языка, ну, я просто все время думаю об этом с... со странным чувством, что на\n\nкультурой, людьми и взглядами. И война, конечно... Господи, ну кто бы... Часть культуры. И, кстати, я подхвачу \nвашу мысль. Война действительно является частью этой культуры, и это, к сожалению, та часть, которая стала очень \nважной для российского общества и для его языка. И в целом она формирует. Главную, формирующую, на мой взг

In [ ]:
answer= model.invoke(final_prompt)
print(answer)

content='В предоставленном контексте тема телефонов не обсуждается. Речь идёт о роли войны как части культуры, её влиянии на российское общество, язык и формирование парадигмы деления на "друга или врага". Если у вас есть другие вопросы по теме, уточните.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 221, 'prompt_tokens': 365, 'total_tokens': 586, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'deepseek/deepseek-r1:free', 'system_fingerprint': None, 'id': 'gen-1755673208-efVTB5NPmSped60IyIoE', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None} id='run--b4d61919-7742-4593-80fd-1694f53647b4-0' usage_metadata={'input_tokens': 365, 'output_tokens': 221, 'total_tokens': 586, 'input_token_details': {}, 'output_token_details': {}}


In [ ]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

In [ ]:
def format_docs(retrieved_docs):
    context_text = '\n\n'.join(doc.page_content for doc in retrieved_docs)
    return context_text

In [ ]:
parallel_chain = RunnableParallel({
    'context': retriever | RunnableLambda(format_docs),
    'question': RunnablePassthrough()
})

In [ ]:
parallel_chain.invoke('what technologies are mentioned in the transcript?')

{'context': 'Привет всем! Я Никита. Я Марина, привет! Мы ведем подкаст «Easy Russian» от Easy Languages, \nподкаст для тех, кто изучает русский язык, несмотря ни на что. Это седьмой выпуск нашего подкаста, \nв котором мы поговорим о том, как война изменила русский язык. Ну, не случайно мы говорим об этом \nименно сейчас. Все, конечно, заняты этой страшной годовщиной, которая приближается. К моменту выхода \nподкаста до нее останется всего два дня, если я не ошибаюсь. Приближается двадцать четвертое февраля,\n\nПривет всем! Я Никита. Я Марина, привет! Мы ведем подкаст «Easy Russian» от Easy Languages, \nподкаст для тех, кто изучает русский язык, несмотря ни на что. Это седьмой выпуск нашего подкаста, \nв котором мы поговорим о том, как война изменила русский язык. Ну, не случайно мы говорим об этом \nименно сейчас. Все, конечно, заняты этой страшной годовщиной, которая приближается. К моменту выхода \nподкаста до нее останется всего два дня, если я не ошибаюсь. Приближается двадцать чет

In [ ]:
parser = StrOutputParser()

In [ ]:
main_chain = parallel_chain | prompt | model | parser

In [ ]:
main_chain.invoke('can you summarize the video?')

'Основной темой обсуждения является влияние войны, начатой Россией 24 февраля 2022 года, на язык и общество. Отмечается, что война не только изменила жизнь людей, но и породила новые слова, отражающие реалии происходящего. Подкаст фокусируется на связи языка с событиями в стране и мире, хотя также затрагивает более широкие вопросы о последствиях конфликта для семьи, общества и будущего. Однако глубокая аналитика этих аспектов пока откладывается, так как основная цель — исследование языковых изменений.'